In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import math

import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch
import pandas as pd
import numpy as np

from model import TDNN

# Data manipulate

In [2]:
import pandas as pd
df_tgt = pd.read_csv('data/target.csv')
df = pd.read_csv('data/data.csv')

/tmp/ipykernel_1639075/1779014282.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/data.csv')


In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_path, target_path):
        
        data_df = pd.read_csv(data_path)
        target_df = pd.read_csv(target_path)
    
        self.data = data_df.iloc[:,2:] # 시간, 수산대교(창원) 데이터 제외
        self.target = target_df.iloc[:,3] # 수위(EL.m)만 갖도록 함


    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return

In [ ]:
import pandas as pd
df_tgt = pd.read_csv('data/target.csv')
df = pd.read_csv('data/data.csv')

In [4]:
df = df.drop(['Unnamed: 0'], axis = 1)
df

,수산대교(창원)_수위,여의리(의령군)_수위,의령군(정암교)_수위,함안군(서출리)_수위,합천군(적포교)_수위,길곡_강수량(mm),대병_강수량(mm),창녕_강수량(mm),청덕_강수량(mm),함안_강수량(mm)
0,1.52,0.57,3.83,4.41,0.63,0.0,0.0,0.0,0.0,0.0
1,1.52,0.57,3.82,4.4,0.63,0.0,0.0,0.0,0.0,0.0
2,1.54,0.57,3.81,4.39,0.62,0.0,0.0,0.0,0.0,0.0
3,1.58,0.57,3.79,4.38,0.61,0.0,0.0,0.0,0.0,0.0
4,1.63,0.57,3.77,4.37,0.60,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
81930,1.22,10.09,0.17,1.82,4.36,0.0,0.0,0.0,0.0,0.0
81931,1.22,10.08,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0
81932,1.22,10.09,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0
81933,1.21,10.1,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0


In [5]:
df_tgt = df_tgt.drop(['Unnamed: 0.1','Unnamed: 0'], axis = 1)
df_tgt

,창녕함안보_수위(EL.m)
0,5.45
1,5.45
2,5.45
3,5.45
4,5.45
...,...
81930,4.89
81931,4.89
81932,4.90
81933,4.90


## Make Sliding Window

In [6]:
def make_sliding_window(data, lag): # data = dataframe, lag = window size

    sliding_lst = [data]

    for i in range(lag):
        sliding_lst.append(data.shift(i+1))
    

    sliding_df = pd.concat(sliding_lst, axis=1)
    
    return sliding_df

In [7]:
sliding_df = make_sliding_window(df, 12)
sliding_df

,수산대교(창원)_수위,여의리(의령군)_수위,의령군(정암교)_수위,함안군(서출리)_수위,합천군(적포교)_수위,길곡_강수량(mm),대병_강수량(mm),창녕_강수량(mm),청덕_강수량(mm),함안_강수량(mm),...,수산대교(창원)_수위,여의리(의령군)_수위,의령군(정암교)_수위,함안군(서출리)_수위,합천군(적포교)_수위,길곡_강수량(mm),대병_강수량(mm),창녕_강수량(mm),청덕_강수량(mm),함안_강수량(mm)
0,1.52,0.57,3.83,4.41,0.63,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.52,0.57,3.82,4.4,0.63,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.54,0.57,3.81,4.39,0.62,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.58,0.57,3.79,4.38,0.61,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.63,0.57,3.77,4.37,0.60,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81930,1.22,10.09,0.17,1.82,4.36,0.0,0.0,0.0,0.0,0.0,...,1.23,10.12,0.18,1.85,4.39,0.0,0.0,0.0,0.0,0.0
81931,1.22,10.08,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0,...,1.24,10.11,0.18,1.85,4.39,0.0,0.0,0.0,0.0,0.0
81932,1.22,10.09,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0,...,1.24,10.11,0.17,1.84,4.39,0.0,0.0,0.0,0.0,0.0
81933,1.21,10.1,0.17,1.81,4.36,0.0,0.0,0.0,0.0,0.0,...,1.22,10.11,0.17,1.84,4.39,0.0,0.0,0.0,0.0,0.0


In [8]:
sliding_df.isnull().sum()

수산대교(창원)_수위     0
여의리(의령군)_수위     0
의령군(정암교)_수위     0
함안군(서출리)_수위     0
합천군(적포교)_수위     0
               ..
길곡_강수량(mm)     12
대병_강수량(mm)     12
창녕_강수량(mm)     12
청덕_강수량(mm)     12
함안_강수량(mm)     12
Length: 130, dtype: int64